# TMA Max Speed

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient

from datetime import timezone
from pathlib import Path


import warnings

warnings.filterwarnings("ignore")

In [ ]:
client = makeEfdClient()

dayObs = ""  # empty for now
file_pattern = f"max_slew_velocity_{dayObs}.png"
plot_folder = Path("./plots")
plot_folder.mkdir(exist_ok=True, parents=True)

## Helper Functions

In [ ]:
def get_events(day_obs, event_maker):
    events = event_maker.getEvents(dayObs)

    # Get lists of slew and track events
    slews = [e for e in events if e.type == TMAState.SLEWING]
    tracks = [e for e in events if e.type == TMAState.TRACKING]
    print(f"Found {len(slews)} slews and {len(tracks)} tracks")

    return events

In [ ]:
async def query_efd(events):
    az = await client.select_time_series(
        topic_name="lsst.sal.MTMount.azimuth",
        fields=["actualPosition", "actualVelocity", "timestamp"],
        start=events[0].begin,
        end=events[-1].end,
    )

    el = await client.select_time_series(
        topic_name="lsst.sal.MTMount.elevation",
        fields=["actualPosition", "actualVelocity", "timestamp"],
        start=events[0].begin,
        end=events[-1].end,
    )

    return az, el

In [ ]:
def get_max_azel_velocity(events, az, el):
    columns = ["seq_num", "begin", "end", "max_az_vel", "max_el_vel"]
    data = []

    for e in events:
        if e.type == TMAState.SLEWING:
            begin = e.begin.datetime.replace(tzinfo=timezone.utc)
            end = e.end.datetime.replace(tzinfo=timezone.utc)

            max_az_vel = az[begin:end]["actualVelocity"].max()
            max_el_vel = el[begin:end]["actualVelocity"].max()

            data_to_append = [e.seqNum, e.begin.iso, e.end.iso, max_az_vel, max_el_vel]
            data.append({col: data_to_append[i] for i, col in enumerate(columns)})

    df = pd.DataFrame(data)
    df = df[df.max_az_vel < 11.0]
    df = df[df.max_el_vel < 11.0]
    df.describe()

    return df

In [ ]:
def plot_histogram(df, dayObs):
    fig, axs = plt.subplots(num=f"slew_histogram_{dayObs}", ncols=2, sharey=True)

    axs[0].hist(df["max_az_vel"], log=True, color="cornflowerblue", ec="white")
    axs[0].set_ylabel("Number of Slews")
    axs[0].set_xlabel("Maximum Velocity\n in Azimuth [deg/s]")
    axs[0].grid(":", alpha=0.25)

    axs[1].hist(df["max_el_vel"], log=True, color="firebrick", ec="white")
    axs[1].set_xlabel("Maximum Velocity\n in Elevation [deg/s]")
    axs[1].grid(":", alpha=0.25)

    fig.suptitle(f"Maximum Slew Velocity on\n{dayObs}")
    fig.tight_layout()
    plt.show()

    filename = str(plot_folder / file_pattern.format(dayObs=dayObs))
    print(filename)
    fig.savefig(filename)

# Single Day Obs Analysis

In [ ]:
dayObs = 20231124

In [ ]:
events = get_events(dayObs, eventMaker)

In [ ]:
az, el = await query_efd(events)

In [ ]:
df = get_max_azel_velocity(events, az, el)

In [ ]:
%matplotlib inline
plot_histogram(df, dayObs)

## Multiple Day Obs Analysis

In [ ]:
day_obs_list = [
    # 20231119,
    # 20231120,
    # 20231121,
    # 20231122,
    # 20231123,
    20231124
]

In [ ]:
%matplotlib inline
eventMaker = TMAEventMaker()

for dayObs in day_obs_list:
    events = get_events(dayObs, eventMaker)
    az, el = await query_efd(events)
    df = get_max_azel_velocity(events, az, el)
    plot_histogram(df, dayObs)